In [ ]:
from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties
import matplotlib.pyplot as pl
import json
import pandas as pd
import pooch
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
climate_response_df = pd.read_csv('../data/fair-calibrations/4xCO2_energy_balance_ebm3.csv')

In [ ]:
climate_response_df#.loc[0, 'C1':'C3']

In [ ]:
aerosol_df = pd.read_csv('../data/smith2021/UKESM1-0-LL.csv', index_col='year')
aerosol_df

In [ ]:
# aerosol indirect parameter calibration; excluding methane oxidation effects
beta = -0.735769463
n_SO2 = 0.037031928
n_BC = 2.84E-31
n_OC = 0.016231426

In [ ]:
erfari = aerosol_df['aprp_ERFariSW'] + aerosol_df['aprp_ERFariLW']

In [ ]:
erfaci = aerosol_df['aprp_ERFaciSW'] + aerosol_df['aprp_ERFaciLW']

In [ ]:
pl.plot(erfari)

In [ ]:
pl.plot(erfaci)

In [ ]:
pl.plot(erfari + erfaci)

In [ ]:
f = FAIR()

In [ ]:
scenarios = ['ssp245']
configs = ['UKESM1-0-LL']
#species = ['BC', 'OC', 'Sulfur', 'Aerosol-radiation interactions', 'Aerosol-cloud interactions']
species = ['OC', 'Aerosol-radiation interactions', 'Aerosol-cloud interactions']

In [ ]:
f.define_time(1850, 2015, 1)
f.define_scenarios(scenarios)
f.define_configs(configs)
f.define_species(*read_properties(species=species))

In [ ]:
f.allocate()

In [ ]:
f.fill_from_rcmip()

In [ ]:
fill(f.climate_configs['ocean_heat_capacity'], climate_response_df.loc[0, 'C1':'C3'])
fill(f.climate_configs['ocean_heat_transfer'], climate_response_df.loc[0, 'kappa1':'kappa3'])
fill(f.climate_configs['deep_ocean_efficacy'], climate_response_df.loc[0, 'epsilon'])
fill(f.climate_configs['gamma_autocorrelation'], climate_response_df.loc[0, 'gamma'])

In [ ]:
initialise(f.forcing, 0)
initialise(f.temperature, 0)

In [ ]:
f.fill_species_configs()

In [ ]:
f.species_configs

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
bc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|BC')&(emis_df['Region']=='World'),'1750':'2014'].values.squeeze()
oc = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|OC')&(emis_df['Region']=='World'),'1750':'2014'].values.squeeze()
so2 = emis_df.loc[(emis_df['Scenario']=='historical')&(emis_df['Variable']=='Emissions|Sulfur')&(emis_df['Region']=='World'),'1750':'2014'].values.squeeze()

In [ ]:
# 1850 baselines
#f.species_configs['baseline_emissions'].loc[dict(specie='Sulfur')] = 4.5444637755469
#f.species_configs['baseline_emissions'].loc[dict(specie='BC')] = 2.57112447874271
f.species_configs['baseline_emissions'].loc[dict(specie='OC')] = 18.2268247866406

In [ ]:
# aerosol forcing response
#f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='Sulfur')] = -0.002388644414646368
#f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='BC')] = 0.0025486764590411104
f.species_configs['erfari_radiative_efficiency'].loc[dict(specie='OC')] = -0.0052

f.species_configs['aci_scale'].loc[dict(config='UKESM1-0-LL')] = beta*0.86
#f.species_configs['aci_shape'].loc[dict(specie='Sulfur')] = n_SO2
#f.species_configs['aci_shape'].loc[dict(specie='BC')] = n_BC
f.species_configs['aci_shape'].loc[dict(specie='OC')] = n_OC

In [ ]:
# UKESM calibrations
f.species_configs['aci_scale']

In [ ]:
f.run()

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')])
pl.plot(erfari)

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')])
pl.plot(erfaci)

In [ ]:
pl.plot(f.timebounds, f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')] + f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')])
pl.plot(erfari+erfaci)

In [ ]:
f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1]

In [ ]:
f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1]

In [ ]:
(
    f.forcing.loc[dict(specie='Aerosol-cloud interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1] +
    f.forcing.loc[dict(specie='Aerosol-radiation interactions', config='UKESM1-0-LL', scenario='ssp245')].values[-1]
)